# Modèle de Réponse aux Promotions

Ce notebook développe un modèle d'apprentissage automatique pour prédire l'impact des promotions sur les ventes et la réponse client.

## Objectifs :
- Analyser l'efficacité des promotions
- Construire un modèle prédictif de réponse aux promotions
- Évaluer les performances du modèle
- Fournir des insights actionnables pour la stratégie promotionnelle

In [3]:
# Importer les bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
import snowflake.connector

plt.style.use('default')
sns.set_palette("husl")

print("Bibliothèques importées avec succès !")

Bibliothèques importées avec succès !


In [4]:
# Se connecter à Snowflake
conn_params = {
    'user': 'workshop_user',
    'password': 'VotreMotDePasse123!',
    'account': 'dnb65599',
    'warehouse': 'ANYCOMPANY_WH',
    'database': 'ANYCOMPANY_LAB',
    'schema': 'ANALYTICS'
}

conn = snowflake.connector.connect(**conn_params)
print("Connecté à Snowflake !")

Connecté à Snowflake !


In [5]:
# Charger les features ML
query = """
SELECT * FROM ANALYTICS.ml_features
"""

df = pd.read_sql(query, conn)
print(f"{len(df)} enregistrements chargés")
print(df.head())
print("\nInfo des données :")
print(df.info())

C:\Users\oroug\AppData\Local\Temp\ipykernel_43820\1519741591.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


989 enregistrements chargés
  TRANSACTION_DATE REGION   AMOUNT HAS_ACTIVE_PROMOTION  DISCOUNT_PERCENTAGE  \
0       2010-01-13   Asia  9086.74                   No                  NaN   
1       2010-01-26   Asia  7151.12                   No                  NaN   
2       2010-01-15   Asia  8105.66                   No                  NaN   
3       2010-04-21   Asia  3461.55                   No                  NaN   
4       2010-07-28   Asia  7817.76                   No                  NaN   

   PROMOTION_DURATION_DAYS FEATURE_MONTH  MONTH  DAY_OF_WEEK  IS_WEEKEND  \
0                      NaN    2010-01-01      1            3           0   
1                      NaN    2010-01-01      1            2           0   
2                      NaN    2010-01-01      1            5           0   
3                      NaN    2010-04-01      4            3           0   
4                      NaN    2010-07-01      7            3           0   

     REGION_CATEGORY AMOUNT_CATEGO

In [7]:
# Prétraitement des données
# Features pour la prédiction
features = ['month', 'day_of_week', 'is_weekend', 'has_promotion', 
           'discount_percentage', 'promotion_duration', 'avg_amount_region_month',
           'transaction_count_region_month']

# Target : transaction à valeur élevée (proxy du succès des promotions)
target = 'high_value_transaction'

# Préparer les données
X = df[features].fillna(0)
y = df[target]

# Mettre à l'échelle les features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Features : {features}")
print(f"Target : {target}")
print(f"Distribution des classes : {y.value_counts(normalize=True)}")

KeyError: "None of [Index(['month', 'day_of_week', 'is_weekend', 'has_promotion',\n       'discount_percentage', 'promotion_duration', 'avg_amount_region_month',\n       'transaction_count_region_month'],\n      dtype='object')] are in the [columns]"

In [ ]:
# Diviser les données
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Ensemble d'entraînement : {X_train.shape[0]} échantillons")
print(f"Ensemble de test : {X_test.shape[0]} échantillons")

In [ ]:
# Entraîner le modèle Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    class_weight='balanced'
)

rf_model.fit(X_train, y_train)

# Faire des prédictions
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

print("Modèle entraîné avec succès !")

In [ ]:
# Évaluation du modèle
print("Rapport de Classification :")
print(classification_report(y_test, y_pred))

print(f"Score ROC AUC : {roc_auc_score(y_test, y_pred_proba):.3f}")

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Valeur Faible', 'Valeur Élevée'],
            yticklabels=['Valeur Faible', 'Valeur Élevée'])
plt.title('Matrice de Confusion')
plt.ylabel('Réel')
plt.xlabel('Prédit')
plt.show()

In [ ]:
# Importance des features
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('Importance des Features pour la Prédiction de Réponse aux Promotions')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

print("Top 5 features les plus importantes :")
print(feature_importance.head())

## Insights Business et Recommandations

### Principaux Résultats :
1. **Impact des Promotions** : [Analyse basée sur les résultats du modèle]
2. **Features Importantes** : [Liste des principaux drivers]
3. **Performance du Modèle** : [Scores ROC AUC, précision, rappel]

### Recommandations :
1. **Stratégie Promotionnelle** : Se concentrer sur les promotions à fort impact
2. **Calendrier** : Timing optimal pour les promotions
3. **Ciblage** : Cibler les clients susceptibles de répondre
4. **Niveaux de Remise** : Pourcentages de remise optimaux

### Prochaines Étapes :
- Déployer le modèle pour prédiction en temps réel
- Test A/B des promotions recommandées
- Surveiller et mettre à jour le modèle régulièrement

In [ ]:
# Fermer la connexion
conn.close()
print("Analyse terminée et connexion fermée !")

# Modèle de Réponse aux Promotions

Ce notebook développe un modèle d'apprentissage automatique pour prédire l'impact des promotions sur les ventes et la réponse client.

## Objectifs :
- Analyser l'efficacité des promotions
- Construire un modèle prédictif de réponse aux promotions
- Évaluer les performances du modèle
- Fournir des insights actionnables pour la stratégie promotionnelle

In [ ]:
# Importer les bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
import snowflake.connector
from snowflake.connector.pandas_tools import pd_read_sql

plt.style.use('default')
sns.set_palette("husl")

print("Bibliothèques importées avec succès !")

In [ ]:
# Se connecter à Snowflake
conn_params = {
    'user': 'workshop_user',
    'password': 'VotreMotDePasse123!',
    'account': 'dnb65599.snowflakecomputing.com',
    'warehouse': 'ANYCOMPANY_WH',
    'database': 'ANYCOMPANY_LAB',
    'schema': 'ANALYTICS'
}

conn = snowflake.connector.connect(**conn_params)
print("Connecté à Snowflake !")

In [ ]:
# Charger les features ML
query = """
SELECT * FROM ANALYTICS.ml_features
"""

df = pd_read_sql(query, conn)
print(f"{len(df)} enregistrements chargés")
print(df.head())
print("\nInfo des données :")
print(df.info())

In [ ]:
# Prétraitement des données
# Features pour la prédiction
features = ['month', 'day_of_week', 'is_weekend', 'has_promotion', 
           'discount_percentage', 'promotion_duration', 'avg_amount_region_month',
           'transaction_count_region_month']

# Target : transaction à valeur élevée (proxy du succès des promotions)
target = 'high_value_transaction'

# Préparer les données
X = df[features].fillna(0)
y = df[target]

# Mettre à l'échelle les features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Features : {features}")
print(f"Target : {target}")
print(f"Distribution des classes : {y.value_counts(normalize=True)}")

In [ ]:
# Diviser les données
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Ensemble d'entraînement : {X_train.shape[0]} échantillons")
print(f"Ensemble de test : {X_test.shape[0]} échantillons")

In [ ]:
# Entraîner le modèle Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    class_weight='balanced'
)

rf_model.fit(X_train, y_train)

# Faire des prédictions
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

print("Modèle entraîné avec succès !")

In [ ]:
# Évaluation du modèle
print("Rapport de Classification :")
print(classification_report(y_test, y_pred))

print(f"Score ROC AUC : {roc_auc_score(y_test, y_pred_proba):.3f}")

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Valeur Faible', 'Valeur Élevée'],
            yticklabels=['Valeur Faible', 'Valeur Élevée'])
plt.title('Matrice de Confusion')
plt.ylabel('Réel')
plt.xlabel('Prédit')
plt.show()

In [ ]:
# Importance des features
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('Importance des Features pour la Prédiction de Réponse aux Promotions')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

print("Top 5 features les plus importantes :")
print(feature_importance.head())

## Insights Business et Recommandations

### Principaux Résultats :
1. **Impact des Promotions** : [Analyse basée sur les résultats du modèle]
2. **Features Importantes** : [Liste des principaux drivers]
3. **Performance du Modèle** : [Scores ROC AUC, précision, rappel]

### Recommandations :
1. **Stratégie Promotionnelle** : Se concentrer sur les promotions à fort impact
2. **Calendrier** : Timing optimal pour les promotions
3. **Ciblage** : Cibler les clients susceptibles de répondre
4. **Niveaux de Remise** : Pourcentages de remise optimaux

### Prochaines Étapes :
- Déployer le modèle pour prédiction en temps réel
- Test A/B des promotions recommandées
- Surveiller et mettre à jour le modèle régulièrement

In [ ]:
# Fermer la connexion
conn.close()
print("Analyse terminée et connexion fermée !")

# Promotion Response Prediction Model

This notebook builds a machine learning model to predict the impact of promotions on sales and customer response.

## Objectives:
- Analyze promotion effectiveness
- Build predictive model for promotion response
- Evaluate model performance
- Provide actionable insights for promotion strategy

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
import snowflake.connector
from snowflake.connector.pandas_tools import pd_read_sql

plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

In [ ]:
# Connect to Snowflake
conn_params = {
    'user': 'workshop_user',
    'password': 'VotreMotDePasse123!',
    'account': 'dnb65599.snowflakecomputing.com',
    'warehouse': 'ANYCOMPANY_WH',
    'database': 'ANYCOMPANY_LAB',
    'schema': 'ANALYTICS'
}

conn = snowflake.connector.connect(**conn_params)
print("Connected to Snowflake!")

In [ ]:
# Load ML features
query = """
SELECT * FROM ANALYTICS.ml_features
"""

df = pd_read_sql(query, conn)
print(f"Loaded {len(df)} records")
print(df.head())
print("\nData info:")
print(df.info())

In [ ]:
# Data preprocessing
# Features for prediction
features = ['month', 'day_of_week', 'is_weekend', 'has_promotion', 
           'discount_percentage', 'promotion_duration', 'avg_amount_region_month',
           'transaction_count_region_month']

# Target: high value transaction (proxy for promotion success)
target = 'high_value_transaction'

# Prepare data
X = df[features].fillna(0)
y = df[target]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Features: {features}")
print(f"Target: {target}")
print(f"Class distribution: {y.value_counts(normalize=True)}")

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
# Train Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    class_weight='balanced'
)

rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

print("Model trained successfully!")

In [ ]:
# Model evaluation
print("Classification Report:")
print(classification_report(y_test, y_pred))

print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_proba):.3f}")

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Low Value', 'High Value'],
            yticklabels=['Low Value', 'High Value'])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
# Feature importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('Feature Importance for Promotion Response Prediction')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

print("Top 5 most important features:")
print(feature_importance.head())

## Business Insights and Recommendations

### Key Findings:
1. **Promotion Impact**: [Analysis based on model results]
2. **Important Features**: [List key drivers]
3. **Model Performance**: [ROC AUC, precision, recall]

### Recommendations:
1. **Promotion Strategy**: Focus on high-impact promotions
2. **Timing**: Optimal timing for promotions
3. **Targeting**: Target customers likely to respond
4. **Discount Levels**: Optimal discount percentages

### Next Steps:
- Deploy model for real-time prediction
- A/B test recommended promotions
- Monitor and update model regularly

In [ ]:
# Close connection
conn.close()
print("Analysis completed and connection closed!")